<a href="https://colab.research.google.com/github/sergioc248/apuntes-inteligencia-artificial/blob/main/Apuntes_EDA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuaderno 3: Manejo de Datos Categóricos

Se dividen en datos nominales y datos ordinales, y su tratamiento es esencial para la preparación del dataset, especialmente en el contexto de modelos de aprendizaje automático.


---

## <font color="red">3.1 Datos Nominales
Los datos nominales no tienen un orden inherente. Ejemplos comunes son colores, ciudades o géneros. El tratamiento de estos datos suele implicar transformarlos en representaciones numéricas comprensibles para los algoritmos de machine learning.

### <font color="blue">3.1.1 Métodos de Codificación para Datos Nominales
#### Codificación One-Hot (One-Hot Encoding):
* Genera una columna binaria para cada categoría, asignando un 1 si la fila pertenece a esa categoría y 0 en caso contrario.
* Es útil para datos sin orden jerárquico.

#### Codificación Binaria:
* Convierte las categorías en representaciones binarias compactas.
* Es útil cuando hay muchas categorías para reducir la dimensionalidad.

#### Codificación de Frecuencia:
* Reemplaza cada categoría con su frecuencia de aparición en los datos.
* Puede ser útil para ciertos modelos donde la frecuencia tiene significado.

#### Otros Métodos Avanzados:
* Codificación de Puntuación (Target Encoding): Usa estadísticas derivadas de las etiquetas objetivo.
* Embeddings de Categorías: Técnicas avanzadas que utilizan representaciones en espacios vectoriales.


---

## <font color="red">3.2 Datos Ordinales
Los datos ordinales tienen un orden jerárquico o relación entre las categorías, como niveles de educación o tamaños de ropa.

### <font color="blue">3.2.1 Codificación de Datos Ordinales

#### Respeto al Orden Jerárquico:
* Las categorías deben asignarse a valores numéricos que reflejen su relación de orden.

#### Mapeo a Escalas Numéricas:
* Ejemplo: "Bajo", "Medio", "Alto" → 1, 2, 3.

---

## 3.3 Ejercicio Práctico con Python
Para este Sesion, trabajaremos con un dataset simulado que contiene tanto datos nominales como ordinales.

####Crear el Dataset




In [74]:
import pandas as pd

# Dataset simulado
data = {
    "ID": [1, 2, 3, 4, 5],
    "Ciudad": ["Madrid", "Barcelona", "Sevilla", "Madrid", "Valencia"],  # Nominal
    "Educación": ["Secundaria", "Primaria", "Universitaria", "Primaria", "Universitaria"],  # Ordinal
    "Género": ["F", "M", "F", "M", "F"],  # Nominal
}
df = pd.DataFrame(data)
print("Dataset original:")
print(df)


Dataset original:
   ID     Ciudad      Educación Género
0   1     Madrid     Secundaria      F
1   2  Barcelona       Primaria      M
2   3    Sevilla  Universitaria      F
3   4     Madrid       Primaria      M
4   5   Valencia  Universitaria      F


## <font color="cyan">3.4 Codificación de Datos Nominales
### 1. Codificación One-Hot


In [75]:
# Usando get_dummies para One-Hot Encoding
df_onehot = pd.get_dummies(df, columns=["Ciudad"], prefix="Ciudad")
print("\nCodificación One-Hot para 'Ciudad':")
print(df_onehot)



Codificación One-Hot para 'Ciudad':
   ID      Educación Género  Ciudad_Barcelona  Ciudad_Madrid  Ciudad_Sevilla  \
0   1     Secundaria      F             False           True           False   
1   2       Primaria      M              True          False           False   
2   3  Universitaria      F             False          False            True   
3   4       Primaria      M             False           True           False   
4   5  Universitaria      F             False          False           False   

   Ciudad_Valencia  
0            False  
1            False  
2            False  
3            False  
4             True  


### 2. Codificación One-Hot con Scikit-learn


In [76]:
from sklearn.preprocessing import OneHotEncoder


# Configuración del codificador
miencoder = OneHotEncoder(sparse_output=False)  # Cambiado de sparse a sparse_output
ciudad_onehot_encoded = miencoder.fit_transform(df[["Ciudad"]])

ciudad_onehot_encoded

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

In [77]:
# Crear un DataFrame con los resultados
onehot_df = pd.DataFrame(ciudad_onehot_encoded, columns=miencoder.get_feature_names_out(["Ciudad"]))
print("\nCodificación One-Hot con Scikit-learn:")
print(onehot_df)



Codificación One-Hot con Scikit-learn:
   Ciudad_Barcelona  Ciudad_Madrid  Ciudad_Sevilla  Ciudad_Valencia
0               0.0            1.0             0.0              0.0
1               1.0            0.0             0.0              0.0
2               0.0            0.0             1.0              0.0
3               0.0            1.0             0.0              0.0
4               0.0            0.0             0.0              1.0


Al hacer OneHotEnconding una de las columnas es "extra" y debo **eliminarla** (cualquiera). <br> En este ejemplo, puedo borrar ciudad Valencia, ya que si todas las ciudades tienen 0. Sé que es porque el usuario es de ciudad Valencia.

In [78]:
onehot_df.drop(columns=["Ciudad_Valencia"],axis=1,inplace=True)
print(onehot_df)

   Ciudad_Barcelona  Ciudad_Madrid  Ciudad_Sevilla
0               0.0            1.0             0.0
1               1.0            0.0             0.0
2               0.0            0.0             1.0
3               0.0            1.0             0.0
4               0.0            0.0             0.0


In [79]:
dfc=df.join(onehot_df)
dfc.drop(columns=["Ciudad"],axis=1,inplace=True)
print(dfc)

   ID      Educación Género  Ciudad_Barcelona  Ciudad_Madrid  Ciudad_Sevilla
0   1     Secundaria      F               0.0            1.0             0.0
1   2       Primaria      M               1.0            0.0             0.0
2   3  Universitaria      F               0.0            0.0             1.0
3   4       Primaria      M               0.0            1.0             0.0
4   5  Universitaria      F               0.0            0.0             0.0


### 3. Codificación de Frecuencia


In [80]:
df.Ciudad.value_counts()

,count
Ciudad,
Madrid,2
Barcelona,1
Sevilla,1
Valencia,1


In [81]:
# Codificación basada en la frecuencia
df["Ciudad_Frecuencia"] = df["Ciudad"].map(df["Ciudad"].value_counts())
print("\nCodificación de Frecuencia para 'Ciudad':")
print(df)



Codificación de Frecuencia para 'Ciudad':
   ID     Ciudad      Educación Género  Ciudad_Frecuencia
0   1     Madrid     Secundaria      F                  2
1   2  Barcelona       Primaria      M                  1
2   3    Sevilla  Universitaria      F                  1
3   4     Madrid       Primaria      M                  2
4   5   Valencia  Universitaria      F                  1


In [82]:
dfSinCiudad=df.drop(columns=["Ciudad"],axis=1)
print(dfSinCiudad)

   ID      Educación Género  Ciudad_Frecuencia
0   1     Secundaria      F                  2
1   2       Primaria      M                  1
2   3  Universitaria      F                  1
3   4       Primaria      M                  2
4   5  Universitaria      F                  1


## <font color="cyan">3.5 Codificación de Datos Ordinales
### 1. Mapeo Directo a Valores Numéricos

In [83]:
# Mapeo manual de las categorías
orden_educacion = {"Primaria": 1, "Secundaria": 2, "Universitaria": 3}
df["Educación_Ordinal"] = df["Educación"].map(orden_educacion)
print("\nCodificación Ordinal para 'Educación':")
print(df)



Codificación Ordinal para 'Educación':
   ID     Ciudad      Educación Género  Ciudad_Frecuencia  Educación_Ordinal
0   1     Madrid     Secundaria      F                  2                  2
1   2  Barcelona       Primaria      M                  1                  1
2   3    Sevilla  Universitaria      F                  1                  3
3   4     Madrid       Primaria      M                  2                  1
4   5   Valencia  Universitaria      F                  1                  3


In [84]:
df.drop(columns=["ID","Educación","Ciudad"],axis=1,inplace=True)

In [85]:
print(df)

  Género  Ciudad_Frecuencia  Educación_Ordinal
0      F                  2                  2
1      M                  1                  1
2      F                  1                  3
3      M                  2                  1
4      F                  1                  3


In [86]:
from sklearn.preprocessing import OrdinalEncoder
codifica_genero=OrdinalEncoder()
df["Género_Ordinal"]=codifica_genero.fit_transform(df[["Género"]])
df.drop(columns=["Género"],axis=1,inplace=True)
df

,Ciudad_Frecuencia,Educación_Ordinal,Género_Ordinal
0,2,2,0.0
1,1,1,1.0
2,1,3,0.0
3,2,1,1.0
4,1,3,0.0


### 2. Usar OrdinalEncoder de Scikit-learn


In [87]:
from sklearn.preprocessing import OrdinalEncoder


# Ejemplo de DataFrame
data = {"Educación": ["Primaria", "Secundaria", "Universitaria", "Primaria", "Secundaria"]}
df = pd.DataFrame(data)

# Configuración del codificador
ordinal_encoder = OrdinalEncoder(categories=[["Primaria", "Secundaria", "Universitaria"]])
df["Educación_Ordinal"] = ordinal_encoder.fit_transform(df[["Educación"]])
print("\nCodificación Ordinal con Scikit-learn:")
print(df)



Codificación Ordinal con Scikit-learn:
       Educación  Educación_Ordinal
0       Primaria                0.0
1     Secundaria                1.0
2  Universitaria                2.0
3       Primaria                0.0
4     Secundaria                1.0


## <font color="cyan">3.7 Uso de Librerías para Guardar Procesos: Joblib

Joblib es ideal para serializar objetos grandes como modelos de machine learning o encoders. Es rápido y eficiente para matrices grandes.

In [88]:
from sklearn.preprocessing import OneHotEncoder
from joblib import dump, load
import numpy as np

# Datos de ejemplo
datos = np.array([["Bogotá"], ["Cali"], ["Medellín"], ["Cali"], ["Bogotá"]])

# Crear y ajustar el encoder
encoder = OneHotEncoder(sparse_output=False)
ciudad=encoder.fit_transform(datos)

# Guardar el encoder en un archivo
dump(encoder, "codificador_ciudad.joblib")
ciudad

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [89]:
# Crear un DataFrame con los resultados
# Get the names of the output columns from the encoder
column_names = encoder.get_feature_names_out(["Ciudad"])

# Crear un DataFrame con los resultados, and use the correct column_names
onehot_df = pd.DataFrame(ciudad, columns=column_names)
print("\nCodificación de la ciudad:")
print(onehot_df)


Codificación de la ciudad:
   Ciudad_Bogotá  Ciudad_Cali  Ciudad_Medellín
0            1.0          0.0              0.0
1            0.0          1.0              0.0
2            0.0          0.0              1.0
3            0.0          1.0              0.0
4            1.0          0.0              0.0


## <font color="yellow">**3.8 Ejercicio: Guardar y Utilizar Procesos de Codificación**

*Va a aparecer en el quiz*

**Vamos a entrenar un modelo de inteligencia, por fin.**


### Cómo usar Scikit-learn

* Importar la librería
* Instancio el modelo "modelo()"
* Utilizo fit_transform para aplicar el modelo al dataframe original
* Exporto la persistencia (constancia de la lógica que se aplicó para la transformación) usando Joblib

In [90]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from joblib import dump

# Dataset simulado
data = {
    "Género": ["F", "M", "F", "M", "F"],
    "Edad": [25, 32, 18, 40, 22],
    "Compró": ["Sí", "No", "No", "Sí", "Sí"],  # Etiqueta
}
df = pd.DataFrame(data)

# Codificar la columna 'Género'
gender_encoder = LabelEncoder()
df["Género"] = gender_encoder.fit_transform(df["Género"])

# Codificar la columna 'Compró' (etiqueta)
compró_encoder = LabelEncoder()
df["Compró"] = compró_encoder.fit_transform(df["Compró"])

# Guardar los encoders para uso posterior
dump(gender_encoder, "gender_encoder.joblib")
dump(compró_encoder, "compró_encoder.joblib")

print("Datos codificados:")
print(df)


Datos codificados:
   Género  Edad  Compró
0       0    25       1
1       1    32       0
2       0    18       0
3       1    40       1
4       0    22       1


* Entrenar el Modelo:


In [91]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from joblib import dump

# Simulación de un DataFrame para pruebas
data = {
    "Género": [0, 1, 0, 1, 0],  # Ejemplo codificado (F -> 0, M -> 1)
    "Edad": [25, 32, 18, 40, 22],
    "Compró": [1, 0, 0, 1, 1],  # Ejemplo codificado (Sí -> 1, No -> 0)
}
df = pd.DataFrame(data)

# Separar las variables independientes (X) y la dependiente (y)
X = df[["Género", "Edad"]]
y = df["Compró"]

# Entrenar el modelo
model = RandomForestClassifier()
model.fit(X, y)

# Guardar el modelo entrenado
dump(model, "random_forest_model.joblib")

print("Modelo guardado correctamente.")


Modelo guardado correctamente.


* Uso del Modelo en Nuevos Datos:


In [92]:
import pandas as pd
from joblib import load

# Nuevos datos (puedes ajustarlos a tus necesidades)
new_data = pd.DataFrame({
    "Género": ["F", "M", "F"],
    "Edad": [28, 34, 21],
})

# Cargar los encoders previamente guardados
gender_encoder = load("gender_encoder.joblib")
compró_encoder = load("compró_encoder.joblib")

# Transformar la columna 'Género' usando el encoder cargado
new_data["Género"] = gender_encoder.transform(new_data["Género"])

# Cargar el modelo entrenado
model = load("random_forest_model.joblib")

# Hacer predicciones con el modelo
X_new = new_data[["Género", "Edad"]]  # Seleccionar las mismas columnas que usaste para entrenar el modelo
predictions = model.predict(X_new)

# Decodificar las predicciones (convertir las etiquetas numéricas a las originales)
decoded_predictions = compró_encoder.inverse_transform(predictions)

# Mostrar las predicciones decodificadas
print("Predicciones para nuevos datos:")
for i, pred in enumerate(decoded_predictions):
    print(f"Nuevo dato {i+1}: Compró = {pred}")


Predicciones para nuevos datos:
Nuevo dato 1: Compró = Sí
Nuevo dato 2: Compró = No
Nuevo dato 3: Compró = Sí


## <font color="red">3.9 Resultados y Atributos de Modelos

Al usar modelos con datos codificados, es importante analizar los atributos del modelo. Por ejemplo, para un modelo de árbol de decisión:
* feature_importances_: Muestra la importancia relativa de cada característica (incluidas las codificadas).
* predict_proba(): Proporciona la probabilidad de cada clase para un conjunto de datos.

Ejemplo


In [94]:
# Importancia de las características
print("Importancia de las características:")
print(model.feature_importances_)

# Probabilidad de predicción
proba = model.predict_proba(new_data)
print("\nProbabilidad de cada clase:")
print(proba)


Importancia de las características:
[0.17881944 0.82118056]

Probabilidad de cada clase:
[[0.1  0.9 ]
 [0.63 0.37]
 [0.26 0.74]]
